In [1]:
import pandas as pd
import googlemaps
from datetime import datetime, timezone
from tqdm import tqdm
import csv
tqdm.pandas()

#Uncomment below when you get your own API key from Google Maps API
#API_key = 'Your own API key'
#gmaps = googlemaps.Client(key=API_key)

In [2]:
# origin_lat, origin_lon: the latitude/longitude of BG
# address: the address of the BG's nearest food pantry or bank.

def compute_travel_time_bg(origin_lat, origin_lon, address):
    
    origin = '%f,%f' % (float(origin_lat), float(origin_lon))
    destination = address
    
    dt = datetime.strptime("13/02/23 14:30", "%d/%m/%y %H:%M") # change with your time   
    dt = dt.replace(tzinfo=timezone.utc)

    result_transit = gmaps.distance_matrix(origin, destination, mode='transit', departure_time=dt)
    result_walk = gmaps.distance_matrix(origin, destination, mode='walking', departure_time=dt)
    
    try:
        if result_transit["rows"][0]["elements"][0]["status"] == "OK":
            if result_walk["rows"][0]["elements"][0]["status"] == "OK":
                duration_transit = result_transit["rows"][0]["elements"][0]['duration']['value'] / 60
                duration_walk = result_walk["rows"][0]["elements"][0]['duration']['value'] / 60
            else:
                duration_transit = result_transit["rows"][0]["elements"][0]['duration']['value'] / 60
                duration_walk = None               
        else:
            if result_walk["rows"][0]["elements"][0]["status"] == "OK":
                duration_walk = result_walk["rows"][0]["elements"][0]['duration']['value'] / 60
                duration_transit = None
            else:
                duration_transit = None
                duration_walk = None
       
        return duration_transit, duration_walk
    
    except IndexError:
        return None, None

1. Example usage of `gmaps.distance_matrix` function

In [7]:
dt = datetime.strptime("13/02/23 11:30", "%d/%m/%y %H:%M")    
dt = dt.replace(tzinfo=timezone.utc)

gmaps.distance_matrix('456 Sunderland Rd, Amherst, MA 01002', 'Lederle Graduate Research Tower, Amherst, MA 01002', mode='transit', departure_time=dt)

{'destination_addresses': ['Lederle Graduate Research Tower, Amherst, MA 01002, USA'],
 'origin_addresses': ['456 Sunderland Rd, Amherst, MA 01002, USA'],
 'rows': [{'elements': [{'distance': {'text': '5.1 km', 'value': 5090},
     'duration': {'text': '15 mins', 'value': 917},
     'fare': {'currency': 'USD', 'text': '$1.50', 'value': 1.5},
     'status': 'OK'}]}],
 'status': 'OK'}

2. Example run of `compute_travel_time` function

: This will create a csv file `bg_transit_adi.csv` in `data` folder, which contains all information about BGs, their nearset FPs, distance, ADI info, state info, and travel time (both by transit and by walking).

In [3]:
food = pd.read_pickle('../data/food_bg20_adi_st.pkl')

`bg_near_pantry` is a dataframe which retrieves each of BGs in `food` dataset and their nearest FPs. 

In [4]:
bg_near_pantry =food.loc[food.groupby('bg_fips')['distance_mi'].idxmin()]
bg_near_pantry = bg_near_pantry[['bg_fips', 'bg_lat', 'bg_lon', 'address', 'distance_mi', 'ADI_NATRANK', 'ADI_STATERNK', 
                        'bg_state', 'address_state']]
bg_near_pantry.reset_index(drop=True)

,bg_fips,bg_lat,bg_lon,address,distance_mi,ADI_NATRANK,ADI_STATERNK,bg_state,address_state
0,010010201001,32.465832,-86.489661,"203 N Court St, Prattville, AL 36067",0.835864,73,5,AL,AL
1,010010201002,32.485873,-86.489672,"203 N Court St, Prattville, AL 36067",1.684913,62,3,AL,AL
2,010010202001,32.480082,-86.474974,"203 N Court St, Prattville, AL 36067",1.065305,83,7,AL,AL
3,010010202002,32.464435,-86.469766,"203 N Court St, Prattville, AL 36067",0.327409,87,7,AL,AL
4,010010203001,32.480175,-86.460792,"203 N Court St, Prattville, AL 36067",1.367486,73,5,AL,AL
...,...,...,...,...,...,...,...,...,...
232730,560430003012,44.011329,-107.953427,"620 Big Horn Ave, Worland, WY 82401",0.529835,76,10,WY,WY
232731,560430003013,44.018328,-107.955039,"620 Big Horn Ave, Worland, WY 82401",0.308140,68,9,WY,WY
232732,560430003021,44.028771,-107.950748,"620 Big Horn Ave, Worland, WY 82401",0.966369,62,8,WY,WY
232733,560430003022,44.012966,-107.937466,"620 Big Horn Ave, Worland, WY 82401",1.192660,58,7,WY,WY


In [ ]:
bg_near_pantry['address'] = bg_near_pantry['address'].str.strip() # strip white space

In [ ]:
with open('../data/bg_transit_adi.csv', 'w', encoding='UTF-8') as fd:
    
    writer = csv.writer(fd)
    writer.writerow(['bg_fips', 'address', 'distance_mi', 'ADI_NATRANK', 'ADI_STATERNK', 'bg_state', 'address_state', 
                     'transit_time', 'walking_time'])        
    
    for i in tqdm(range(len(bg_near_pantry))):
        
        bg_fips = bg_near_pantry['bg_fips'].iloc[i] # FIPS
        bg_lat = bg_near_pantry['bg_lat'].iloc[i] # Latitude
        bg_lon = bg_near_pantry['bg_lon'].iloc[i] # Longitude
        fp_address = bg_near_pantry['address'].iloc[i] # address
        bg_distance = bg_near_pantry['distance_mi'].iloc[i] # distance between BG and nearest FP
        bg_adi = bg_near_pantry['ADI_NATRANK'].iloc[i] # BG's national ADI
        bg_state_adi = bg_near_pantry['ADI_STATERNK'].iloc[i] # BG's state ADI
        bg_state = bg_near_pantry['bg_state'].iloc[i] # BG's state
        fp_state = bg_near_pantry['address_state'].iloc[i] # FP's state
        
        (transit_t, walk_t) = compute_travel_time_bg(bg_lat, bg_lon, fp_address)
        
        writer.writerow([bg_fips, fp_address, bg_distance, bg_adi, bg_state_adi, bg_state, fp_state, transit_t, walk_t])

fd.close()

Now, you will get the dataset to conduct accessibility analysis! 